# 0. 드라이브 - 코랩 연결

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. 토크나이즈된 .arrow 데이터 불러오기

In [32]:
from datasets import load_from_disk

train_ds = load_from_disk('/content/drive/MyDrive/DILAB/HJ/News_DILAB/train_tokenized')
valid_ds = load_from_disk('/content/drive/MyDrive/DILAB/HJ/News_DILAB/valid_tokenized')

print(len(train_ds["input_ids"]))
print(valid_ds)

# "input_ids" : 토크나이저가 텍스트를 분해해 만든 토큰 정수 ID 배열
# "attention_mask" : 어떤 토큰을 실제로 계산에 반영할지 표시하는 마스크
  # ㄴ> 1: 유효 토큰, 0: 패딩(무시)

2048
Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 32
})


# 2. labels(학습을 위한 정답지) 생성

In [25]:
IGNORE_INDEX = -100 # CE Loss의 ignore_index 값은 -100이다.

# 해당 train/valid 데이터셋에 각각의 labels를 추가해주는 함수
def add_labels_batched(batch):
  ids_batch = batch["input_ids"]
  # attention_mask를 가져오는데, 만약 해당 값이 없다면 default 값으로 input_ids 길이만큼의 "1"로 채워진 batch형 배열 반환
  am_batch = batch.get("attention_mask", [[1]*len(ids) for ids in ids_batch])

  # attention_mask에서 1이 아닌 자리는 IGNORE_INDEX로 채우고, 1인 자리는 input_ids의 값을 복사한다.
  # 해당 작업을 배치 단위로 진행한다.
  labels_batch = [
      [tok if m==1 else IGNORE_INDEX for tok, m in zip(ids, am)]
      for ids, am in zip(ids_batch, am_batch)
  ]
  return {"labels": labels_batch}

# 각각의 데이터셋에 "labels"를 추가함
  # num_proc : 병렬 처리에 사용할 프로세스 개수
  # batched : 해당 map작업을 배치 단위로 하는지 여부 설정
# ㄴ> map 함수는 해당 함수 작업을 수행한 결과를 해당 데이터셋에 추가한 값을 반환한다.
train_ds = train_ds.map(add_labels_batched, batched=True, num_proc=4)
valid_ds = valid_ds.map(add_labels_batched, batched=True, num_proc=4)


# 해당 데이터셋에는 "text" 컬럼이 있는데, 해당 컬럼은 학습 과정에서 필요 없기 때문에 제거한다
train_ds = train_ds.remove_columns("text")
valid_ds = valid_ds.remove_columns("text")

Map (num_proc=4):   0%|          | 0/280 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/32 [00:00<?, ? examples/s]

# 3. 모델에 8비트 양자화 + LoRA 적용

In [26]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, default_data_collator
from transformers import AutoTokenizer  # 임베딩 리사이즈 & 생성 시에만 사용
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

model_path = "/content/drive/MyDrive/DILAB/llama3-Korean-Bllossom-8B"

tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)
# PAD 토큰이 없다면, EOS 토큰으로 해당 역할 대체
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token


# 1. 8비트 양자화 로드
bnb_config = BitsAndBytesConfig(
    load_in_8bit = True,
    # fp16 대신 bfloat16을 연산 타입으로 사용 (bfloat16이 fp16보다 안정적임)
    bnb_8bit_compute_dtype = torch.bfloat16
)

# 2. 8비트 양자화 적용한 모델 로드
# AutoModelForCausalLM으로 8비트 양자화를 적용하여 저장해둔 모델(라마 3)을 로드한다.
base_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config = bnb_config, # 모델을 8-bit 양자화 하여 GPU 메모리를 절약 (Hugging Face + bitsandbytes 라이브러리로 작동함)
    device_map = "auto", # "auto": 모델을 사용 가능한 장치에 자동으로 분산해서 올려준다 (GPU, CPU에 자동 분산) / "cuda' : 자동 분산하지 않고 단일 GPU 사용
    torch_dtype = torch.bfloat16 # torch의 데이터 타입을 bfloat16으로 바꿔줌
)

# 3.base_model의 입력 임베딩 테이블 resize
  # naver_news_tokeniziner 코드에서 텍스트에 "<title></title>, <body></body>" 스페셜 토큰을 적용해준 뒤 Tokenizer에도
  # 해당 스페셜 토큰들을 추가해주었었기 때문에 vocab(해당 Tokenizer의 어휘 사전 목록 길이)가 커졌다.
  # 따라서, 모델의 임베딩 테이블의 크기 또한 새로운 vocab으로 설정해주지 않으면 학습 중에 해당 토큰 id를 임베딩에서 찾을
  # 때 IndexError(범위 초과)가 날 수 있다.
    # ㄴ> 토크나이징 결과 숫자는 해당 토큰의 ID(Tokenizer, Model의 어휘 사전(vocab)에서의 위치)이다.
    # ㄴ> 해당 토큰 ID로 임베딩 테이블의 vector값을 찾는 것이다. (이 벡터값이 학습 중에 업데이트되는 가중치이다.)
    # ㄴ> 그렇기 때문에, resize_token_embedding()을 해주지 않으면, 임베딩 테이블에서 indexError가 날 수 있는 것이다.
base_model.resize_token_embeddings(len(tokenizer)) # base_model의 임베딩 행 개수를 늘려, 새 토큰들용 임베딩을 추가해야됨
  # ㄴ> model.resize_token_embeddings() : 모델의 입력 임베딩 테이블을 새 vocab 크기에 맞게 리사이즈하는 함수
    # 늘어나는 행 => 랜덤 초기화. 학습 중에 업데이트 된다
    # 줄이는 행 => 뒤쪽 행을 잘라냄

# 4. K/V 캐시(past_key_values) 사용 여부를 끈다
  # 학습 중에는 gradient checkpointing과 충돌/경고가 나거나, 캐시 유지가 메모리를 더 먹을 수 있음
  # 따라서, 보통 학습할 때는 K/V 옵션을 끄며, 추론&생성 시에는 킨다
  # 효과 : 학습 중 경고 제거, 메모리 절약
base_model.config.use_cache = False

# 5. Gradient(Activation) Checkpointing을 킨다
 # Gradient(Activation) Checkpointing : 순전파 때 중간 활성값을 전부 저장하지 않고, 역전파 때 일부를 재계산해서 VRAM을 크게 절약하는 기법
 # 장점: 메모리 사용량 감소 / 단점 : 연산량 증가
base_model.gradient_checkpointing_enable()


# 6. LoRA 적용

  # 6-1) 양자화된 모델을 LoRA 학습이 가능한 상태로 설정
# prepare_model_for_kbit_training(model) : 8비트/4비트 양자화된 모델을 LoRA로 학습 가능한 상태로 안전하게 세팅해주는 함수
# LoRA를 모델에 붙이기 전에 해당 작업을 해주는 것이 바람직함
model_ready = prepare_model_for_kbit_training(base_model)

  # 6-2) LoRA를 적용할 선형층 지정
# "q_proj","k_proj","v_proj","o_proj" => 어텐션 모듈의 Q/K/V/O 투영 선형층
# "gate_proj","up_proj","down_proj" => FFN(MLP)의 게이트/상향/하향 선형층
target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"]  # LLaMA3 계열 흔한 선택

  # 6-3) LoRA 설정값 지정 (LoraConfig)
# LoraConfig() : LoRA 방식의 설정값들을 지정하는 함수
peft_conf = LoraConfig(
    r=16, # -> 랭크(학습할 저랭크 행렬의 차원 수). 클수록 표현력&파라미터 수 증가, VRAM/연산 증가 (보통 8~64 범위로 지정함)
    lora_alpha=32,  # -> scaling factor로, 최종 결과는 "lora_alpha / r"로 스케일링 됨(모델 성능에 영향을 줄 수 있음)
                    # -> 앞서 만든 "A @ B" 저랭크 행렬이 원래 weight W에 비해 너무 작은 값이 될 수 있으므로, 결과에 스케일링 계수(lora_alpha/r)를 곱해서 크기를 조절해주는 것.
    lora_dropout=0.05, # -> 과적합 방지를 위한 드롭아웃 확률 설정
    bias="none", # -> 기존 모델의 bias(편향) 파라미터를 학습에 포함시킬 지 여부 지정 ("lora_only", "all" 옵션도 존재함)
    task_type="CAUSAL_LM", # -> 작업 종류(저장/로드/일부 내부 로직에서 적절한 타입을 )
    target_modules=target_modules # -> 어떤 선형층에 LoRA를 주입할지 선택
)

  # 6-4) LoRA 모듈 주입
# model_ready의 지정된 target_modules 선형층마다 "LoRA 어댑터(저랭크 A/B 행렬)"가 붙는다
# get_peft_model() : LoRA 설정에 맞게 기존 모델의 일부 모듈(q_proj, v_proj 등)에 LoRA 계층을 삽입해서, 전체 모델을 래핑(wrap)해준다.
model = get_peft_model(model_ready, peft_conf)


ImportError: Using `bitsandbytes` 8-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

# 4. N 에폭만큼 학습 진행

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch.nn.utils as nn_utils
from contextlib import nullcontext
from tqdm.auto import tqdm
import math

# 1. 배치 단위로 데이터 공급해주기 위해 DataLoader 사용
def collate_fn(batch):
    return {
        "input_ids":      torch.tensor([b["input_ids"]      for b in batch], dtype=torch.long),
        "attention_mask": torch.tensor([b["attention_mask"] for b in batch], dtype=torch.long),
        "labels":         torch.tensor([b["labels"]         for b in batch], dtype=torch.long),
    }

# 미니 배치들을 만드는 과정이다. 한 번의 iteration마다 샘플을 batch_size만큼 개를 꺼내온다는 뜻
# 데이터셋의 각 컬럼은 [X * Y]처럼 생겼는데, 해당 데이터에서 batch_size만큼의 행(샘플)들을 꺼내와서 배치화한다는 것이다.
train_loader = DataLoader(train_ds, batch_size=2, shuffle=True,  collate_fn=collate_fn, pin_memory=True)
valid_loader = DataLoader(valid_ds, batch_size=2, shuffle=False, collate_fn=collate_fn, pin_memory=True)



# 2. Optimizer 설정(Peft 사용 시, base_model의 trainable 파라미터만 학습된다.)
  # 옵티마이저는 기울기(미분값)를 사용하여 파라미터를 업데이트하는 역할을 한다.

# optimizer = AdamW(model.parameters(), lr=2e-4) # -> Full 파인튜닝(모든 가중치 학습)을 할 때
  # ㄴ> Pytorch의 AdamW는 grad가 없는 파라미터는 step()에서 건너뛰지만, 불필요한 파라미터 레퍼런스를 옵티마이저가 들고 있게 되므로, 오버헤드가 증가하게 된다.

optimizer = AdamW([p if p.requires_grad for p in model.parameters()], lr-2e-4)
  # ㄴ> LoRA는 본체 가중치는 동결하고 LoRA 어댑터만 학습하기 때문에, 현재 model의 파라미터 중 학습할 파라미터만 모아서 옵티마이저에게 넘겨주어야 한다
  # ㄴ> requires_grad 속성으로 해당 파라미터가 학습할 파라미터인지 아닌지 확인한다.


# 3. GPU 사용 준비
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 4. bf16 autocast (지원 GPU면 on하도록 하는 설정)
use_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8  # Ampere+
amp_ctx = (torch.autocast(device_type="cuda", dtype=torch.bfloat16) if use_bf16 else nullcontext())
  # ㄴ> torch.autocast() 함수가 return한 객체는 "컨텍스트 매니저 객체"이다.
  # ㄴ> with와 함께 사용 시, 블록 내 연산을 "자동 혼합 정밀(AMP)"로 돌려서 속도&VRAM을 절약할 수 있다,
  # ㄴ> autocast 미지원 시, "nullcontext()"로 아무 것도 하지 않는 컨텍스트(fp32)로 동작한다.


# 5. 에폭 설정
epochs = 2
grad_accum = 8  # 몇개의 미니 배치가 모이면 optimizer로 한번에 기울기 업데이트를 할지 정하는 변수


# 6. 학습 시작
global_step = 0
for epoch in range(epochs):
  # 해당 모델을 "학습 모드"로 설정
  model.train()

  total_loss = 0

  # tqdm : 진행 상황을 bar로 보여주는 라이브러리
  running = 0.0
  loop = tqdm(train_loader, desc=f"Epoch {epoch+1}") # 데이터를 batch 단위로 반환해주는 Dataloader로 설정한 train_loader를 설정해준다.
        #  ㄴ> tqdm은 원래 iterable인 DataLoader를 감싼 iterable 객체이므로, "for batch in tqdm_obj"로 처럼 batch를 꺼내올 수 있다.

  optimizer.zero_grad(set_to_none=True)

  # DataLoader가 미니 배치로 나누어서 데이터들을 준다.
  # GPU의 한계 때문에 모든 데이터를 한번에 올리지 않고, 여러 개의 미니 배치로 데이터를 나눈 후 gradient 누적을 통해
  # 모든 데이터를 한번에 계산한 것과 같은(비슷한) 효과를 내도록 하는 것
    # grad_accum은 몇개의 배치를 묶어서 한 번 업데이트할 지를 지정하는 변수
  for step, batch in enumerate(loop, start=1): # enumerate는 매 반복마다 (인덱스, 값) 쌍을 뱉기 때문에 step, batch로 모은다
      # ㄴ> step : 만들어질 수 있는 총 배치의 개수
      batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        # ㄴ> 딕셔너리 컴프리헨션 -> {키식: 값식 for k, v in ...}
        # ㄴ> 기존 batch 딕셔너리의 모든 entry 쌍을 돌면서 값을 device(GPU)로 이동시킨다.

      # autocast가 return한 AMP 컨텍스트 객체로 아래 블록의 작업을 수행한다.
      with amp_ctx:
          out = model(**batch)           # CausalLM: loss 포함
                  # ㄴ> 딕셔너리 언패킹 : batch 내부의 여러 컬럼들
                  # {"input_ids" : X, "attention_mask" : Y, "labels" : Z}를
                  # model(input_ids=X, attention_mask=Y, labels=Z)로 키워드 인자로 풀어 넣는다.
          loss = out.loss / grad_accum
            # ㄴ> 그라디언트 누적을 위해, 각 배치의 loss값을 grad_accum으로 나눈 뒤에, 해당 loss값을 이용하여 역전파를 수행한다
            # ㄴ> loss값을 grad_accum으로 나누는 이유는 N개의 미니 배치의 평균 기울기를 만들기 위해서이다.

      loss.backward()
        # ㄴ> .backward()

      if step % grad_accum == 0:
          nn_utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
          optimizer.step()
          optimizer.zero_grad(set_to_none=True)
          global_step += 1

      running += loss.item() * grad_accum
      if step % grad_accum == 0:
          loop.set_postfix(loss=running/grad_accum)
          running = 0.0






